<hr style="height:3px;border:none;color:#333;background-color:#333;" />
<img style=" float:right; display:inline" src="http://opencloud.utsa.edu/wp-content/themes/utsa-oci/images/logo.png"/>

### **University of Texas at San Antonio** 
<br/>
<br/>
<span style="color:#000; font-family: 'Bebas Neue'; font-size: 2.5em;"> **Open Cloud Institute** </span>

<hr style="height:3px;border:none;color:#333;background-color:#333;" />

### Lesson 2: MPI


### Swanand Mhalagi, Research Assistant 
#### swanand.mhalagi@utsa.edu 

**Open Cloud Institute, University of Texas at San Antonio, San Antonio, Texas, USA**
<hr style="height:3px;border:none;color:#333;background-color:#333;" />

### Point to Point Communication:
As tradition has it, we will introduce you to MPI programming using a variation on the standard hello world program: your first MPI python program will be the Hello World program for multiple processes. The source code is as follows:

### Send(...) and Recv(...) Process:


Comm.*Send*(buf, dest = 0, tag = 0) Performs a basic send. 

This send is a point-to-point communication. It sends information from exactly one process to exactly one other process.



        Comm (MPI comm) – communicator we wish to query

        buf (choice) – data to send

        dest (integer) – rank of destination

        tag (integer) – message tag 



Comm.*Recv*(buf, source = 0, tag = 0, Status status = None) Basic point-to-point receive of data 


        Comm (MPI comm) – communicator we wish to query

        buf (choice) – initial address of receive buffer (choose receipt location)

        source (integer) – rank of source

        tag (integer) – message tag

        status (Status) – status of object 


In [4]:
%%bash
cat <<'EOF'> passRandomDraw.py
#passRandomDraw.py
import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

randNum = numpy.zeros(1)

if rank == 1:
        randNum = numpy.random.random_sample(1)
        print "Process", rank, "drew the number", randNum[0]
        comm.Send(randNum, dest=0)

if rank == 0:
        print "Process", rank, "before receiving has the number", randNum[0]
        comm.Recv(randNum, source=1)
        print "Process", rank, "received the number", randNum[0]
EOF

To demonstrate that the code is working the way we expect, we have the drawing process print the value that it drew. Then we have the receiving process print the value of the vairable into which we will receive to show that it is zero, and then we print the value that it receives after the call to *Recv*. *Send* and *Recv* are referred to as blocking functions. That is, if a process calls *Recv*, it will sit idle until it has received a message from a corresponding Send before it will proceeed. See the appendix for the corresponding non-blocking functions *Isend* and *Irecv* (I stands for immediate). In essence, Irecv will return immediately.

In [5]:
!mpiexec -n 2 -mca btl ^openib python passRandomDraw.py

Process 1 drew the number 0.510828594296
Process 0 before receiving has the number 0.0
Process 0 received the number 0.510828594296


*Send* and *Recv* are referred to as blocking functions. That is, if a process calls *Recv*, it will sit idle until it has received a message from a corresponding *Send* before it will proceeed. See the appendix for the corresponding non-blocking functions *Isend* and *Irecv* (I stands for immediate). In essence, *Irecv* will return immediately. If it did not receive its message it will indicate to the system that it will be receiving a message, proceed beyond the *Irecv* to do other useful work, and then check back later to see if the message has arrived. This can be used to dramatically improve performance. On a *Recv* you do not always need to specify the source. Instead, you can allow the calling process to accept a message from any process that happend to be sending to the receiving process. This is done by setting source to a predefined MPI constant, *source=ANY_SOURCE* (note that you would first need to import this with from *mpi4py.MPI import ANY_SOURCE* or use the syntax *source=MPI.ANY_SOURCE*).